In [1]:
%pwd

'd:\\ML_Proj\\End-to-End-ML-Project-with-MLFlow\\research'

In [2]:
import os

os.chdir('../')

In [3]:
%pwd

'd:\\ML_Proj\\End-to-End-ML-Project-with-MLFlow'

In [4]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/manirathinam21/End-to-End-ML-Project-with-MLFlow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="manirathinam21"
os.environ["MLFLOW_TRACKING_PASSWORD"]="758a12655f75625c15f39939bd1fda36c0cc2384"

In [24]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    target_column: str
    mlflow_uri: str
    all_params: dict


In [25]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:

    def __init__ (self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH,
        schema_filepath= SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) ->ModelEvaluationConfig:

        config= self.config.model_evaluation
        params= self.params.ElasticNet
        schema= self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config= ModelEvaluationConfig(
            root_dir=  config.root_dir,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            all_params= params,
            mlflow_uri= "https://dagshub.com/manirathinam21/End-to-End-ML-Project-with-MLFlow.mlflow")

        return model_evaluation_config

In [26]:
import os
import pandas as pd
import numpy as np
import json
import joblib
import mlflow 
import mlflow.sklearn
from pathlib import Path
from urllib.parse import urlparse
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from mlProject import logger


class ModelEvaluation:
    def __init__ (self, config:ModelEvaluationConfig ):
        self.config= config

    def eval_metrics(self, actual, pred):
        rmse= np.sqrt(mean_squared_error(actual, pred))
        mae= mean_absolute_error(actual, pred)
        r2= r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data= pd.read_csv(self.config.test_data_path)
        model= joblib.load(self.config.model_path)

        test_x= test_data.drop([self.config.target_column], axis=1)
        test_y= test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store= urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            pred_y= model.predict(test_x)
            (rmse, mae, r2)= self.eval_metrics(test_y, pred_y)

            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            with open(Path(self.config.metric_file_name), "w") as json_file:
                json.dump(scores, json_file) 
            
            
            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")


In [27]:
try:
    config= ConfigurationManager()
    model_evaluation_config= config.get_model_evaluation_config()
    model_evaluation= ModelEvaluation(config= model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2023-08-17 14:43:25,522: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-17 14:43:25,526: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-17 14:43:25,534: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-17 14:43:25,537: INFO: common: created directory at: artifacts]
[2023-08-17 14:43:25,541: INFO: common: created directory at: artifacts/model_evaluation]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2023/08/17 14:43:44 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 3
Created version '3' of model 'ElasticnetModel'.
